In [30]:
import pandas as pd
import numpy as np
import os

In [31]:
input_file = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting"

In [32]:
# company_files = os.listdir(input_file)

# for company in company_files:
#     if company == "Common_Parameters":
#         continue
#     else:
#         folder_path = os.path.join(input_file, company)
        
#         company_folders = os.listdir(folder_path)
        
#         for folders in company_folders:
#             if folders == "Pruned_Excel":
#                 new_folder_path = os.path.join(folder_path, "Pruned_Excel", "Final_Parameters")
                
#                 excel_files_list = os.listdir(new_folder_path)
                
#                 for excel in excel_files_list:
#                     expected_file = f"{company}_Cleaned_Data_Updated.xlsx"
#                     if excel == expected_file:
#                         excel_file = os.path.join(new_folder_path, expected_file)
                        
#                         dfs = pd.read_excel(excel_file, sheet_name=None)
                        
#                         # print(f"Sheets in {excel_file}: {list(dfs.keys())}")
#                         bfs = dfs.get("Balance_Sheet") 
#                         cfs = dfs.get("Cash_Flow")
#                         pls = dfs.get("Profit_Loss") 
#                         qua = dfs.get("Quarterly")
#                         rat = dfs.get("Ratio")
                        
#                         categories = [("Balance_Sheet", bfs), ("Cash_Flow", cfs), 
#                                   ("Profit_Loss", pls), ("Quarterly", qua), ("Ratio", rat)]
                    
#                         for name, cat in categories:
#                             if cat is not None:  # Ensure the sheet exists before accessing .shape
#                                 print(f"Sheet: {name}, Shape: {cat.shape}")
#                             else:
#                                 print(f"Sheet: {name} not found in {excel_file}")
    


In [33]:
company_files = os.listdir(input_file)

for company in company_files:
    if company == "Common_Parameters":
        continue

    folder_path = os.path.join(input_file, company)
    
    if not os.path.isdir(folder_path):
        continue  # Skip if it's not a directory

    company_folders = os.listdir(folder_path)

    for folders in company_folders:
        if folders == "Pruned_Excel":
            new_folder_path = os.path.join(folder_path, "Pruned_Excel", "Final_Parameters")
            
            if not os.path.isdir(new_folder_path):  # Ensure the folder exists
                continue
            
            excel_files_list = os.listdir(new_folder_path)

            for excel in excel_files_list:
                expected_file = f"{company}_Cleaned_Data_Updated.xlsx"
                if excel == expected_file:
                    excel_file = os.path.join(new_folder_path, expected_file)
                    
                    # Read all sheets
                    dfs = pd.read_excel(excel_file, sheet_name=None)
                    
                    # Process each sheet
                    for sheet_name, df in dfs.items():
                        if df is None or df.empty:
                            print(f"Skipping empty sheet: {sheet_name}")
                            continue
                        
                        # Keep the first column (string) as is
                        first_col = df.iloc[:, 0]  # Store the first column
                        df_numeric = df.iloc[:, 1:]  # Get only numeric columns

                        # Step 1: Replace all 0s with NaN
                        df_numeric.replace(0, np.nan, inplace=True)

                        # Step 2 & 3: Fill NaNs with mean if count is less than half, else keep as is
                        for column in df_numeric.columns:
                            nan_count = df_numeric[column].isna().sum()
                            total_rows = len(df_numeric)
                            
                            if nan_count < (total_rows / 2):  # Less than half -> Fill with mean
                                mean_value = df_numeric[column].mean(skipna=True)  # Compute mean without NaNs
                                mean_value = round(mean_value, 3)  # Round to 3 decimal places
                                df_numeric[column].fillna(mean_value, inplace=True)
                            # If more than half -> Keep NaNs as is

                        # Step 5: Convert remaining NaNs back to 0
                        df_numeric.fillna(0, inplace=True)

                        # Merge back the first column with the modified numeric columns
                        df_final = pd.concat([first_col, df_numeric], axis=1)

                        # Save the modified sheet back
                        dfs[sheet_name] = df_final

                    # Save the modified Excel file
                    output_file = os.path.join(new_folder_path, f"{company}_semi_final.xlsx")
                    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
                        for sheet_name, df in dfs.items():
                            df.to_excel(writer, sheet_name=sheet_name, index=False)

                    print(f"Processed and saved: {output_file}")

C:\Users\sharm\AppData\Local\Temp\ipykernel_11804\1909342369.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_numeric[column].fillna(mean_value, inplace=True)
C:\Users\sharm\AppData\Local\Temp\ipykernel_11804\1909342369.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

Processed and saved: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting\3i Infotech Ltd\Pruned_Excel\Final_Parameters\3i Infotech Ltd_semi_final.xlsx


C:\Users\sharm\AppData\Local\Temp\ipykernel_11804\1909342369.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_numeric[column].fillna(mean_value, inplace=True)
C:\Users\sharm\AppData\Local\Temp\ipykernel_11804\1909342369.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

Processed and saved: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting\Adroit Infotech Ltd\Pruned_Excel\Final_Parameters\Adroit Infotech Ltd_semi_final.xlsx


C:\Users\sharm\AppData\Local\Temp\ipykernel_11804\1909342369.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_numeric[column].fillna(mean_value, inplace=True)
C:\Users\sharm\AppData\Local\Temp\ipykernel_11804\1909342369.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

...

Processed and saved: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting\WEP Solutions Ltd\Pruned_Excel\Final_Parameters\WEP Solutions Ltd_semi_final.xlsx
Processed and saved: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting\Wipro Ltd\Pruned_Excel\Final_Parameters\Wipro Ltd_semi_final.xlsx


C:\Users\sharm\AppData\Local\Temp\ipykernel_11804\1909342369.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_numeric[column].fillna(mean_value, inplace=True)
C:\Users\sharm\AppData\Local\Temp\ipykernel_11804\1909342369.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

Processed and saved: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting\Xchanging Solutions Ltd\Pruned_Excel\Final_Parameters\Xchanging Solutions Ltd_semi_final.xlsx
Processed and saved: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting\Xtglobal Infotech Ltd\Pruned_Excel\Final_Parameters\Xtglobal Infotech Ltd_semi_final.xlsx


C:\Users\sharm\AppData\Local\Temp\ipykernel_11804\1909342369.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_numeric[column].fillna(mean_value, inplace=True)
C:\Users\sharm\AppData\Local\Temp\ipykernel_11804\1909342369.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

Processed and saved: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting\Zensar Technologies Ltd\Pruned_Excel\Final_Parameters\Zensar Technologies Ltd_semi_final.xlsx
